In [5]:
# ======================================================================
# Программа распознает лицо (основываячь на трех известных: меня, боуи и кодзимы) 
# и пишет кто это и насколько лицо отдаленно от эталонного (точность определения)
# ======================================================================

import face_recognition
import cv2
from PIL import Image, ImageDraw
import numpy as np

In [6]:
# Загрузим изображения, распознаем на них лица и получим их кодировку

image_of_daniil = face_recognition.load_image_file('daniil.jpg')
daniil_face_encoding = face_recognition.face_encodings(image_of_daniil)[0]

image_of_kojima = face_recognition.load_image_file('buryat.jpg')
kojima_face_encoding = face_recognition.face_encodings(image_of_kojima)[0]

image_of_bowie = face_recognition.load_image_file('bowie.jpg')
bowie_face_encoding = face_recognition.face_encodings(image_of_bowie)[0]

# Создадим списки наших лиц и их имен
known_face_encodings = [daniil_face_encoding,
                        kojima_face_encoding,
                        bowie_face_encoding]

known_face_names = ["Eto ya",
                    "Kojima",
                    "Bowie"]

# Проиницилизируем переменную
process_this_frame = True

# Подключим нашу вебкамеру и запишем ее в переменную
video_capture = cv2.VideoCapture(0)

# Проинициализруем переменную (для входа во вложенный цикл)
process_this_frame = True



In [7]:
# Бесконечный цикл для потоковой обработки видеоряда с камеры
while True:
    # получим и запишем кадр видеоптока
    ret, frame = video_capture.read()

    # Для более быстрой обработки измениним размер в 1/4 раза 
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Конвертируем цветвоую схему получаемого изображения
    rgb_small_frame = small_frame[:, :, ::-1]

    # Мы входим в цикл только каждый 2 раз, так как переменная process_this_frame 
    # менят значение с True на False. Сделанно это, чтобы уменьшить нагрузку на компьютер 
    if process_this_frame:
        # Записываем локацию лица с кадра видеопотока
        face_locations = face_recognition.face_locations(rgb_small_frame)
        # Кодируем лицо
        face_encodings = face_recognition.face_encodings(rgb_small_frame, 
                                                        face_locations)

        face_names = []
        face_matches=[]
        
        for face_encoding in face_encodings:
            # Сравниваем кодировки лиц с кадра с лицом из наших известных лиц
            matches = face_recognition.compare_faces(known_face_encodings, 
                                                     face_encoding)
            
            # Проинициализируем имя и точность найденного лица по-умолчанию. 
            name = "Unknown"
            best_match = 0
            
            # Определим дистанцию в занчениях найденного лица и заготовленного списка лиц 
            face_distances = face_recognition.face_distance(known_face_encodings, 
                                                            face_encoding)
            
            # Найдем минимальную дистанцию и индекс соотв ей
            best_match_index = np.argmin(face_distances)
 
            if matches[best_match_index]:
                # Пишем имя соответсвуюшее лицу
                name = known_face_names[best_match_index]
                best_match=1-face_distances[best_match_index]
                   
            face_matches.append(best_match)
            face_names.append(name)
   
    # Инверитруем значение переменной process_this_frame, чтобы слудующий кадр не анализировался
    process_this_frame = not process_this_frame

    # Отобразим информацию о лицах на экран
    for (top, right, bottom, left), name, match in zip(face_locations, 
                                                        face_names,
                                                        face_matches):
        # Умножим координаты на 4, так как до этого уменьшали размер фрейма на 4
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Выделим лицо кружком
        center=(int((left+right)/2), int((top+bottom)/2))
        cv2.circle(frame, center, 90, (0, 0, 255), 2)

        # Выведем сопутсующие подписи
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 20), font, 1.0, (255, 255, 255), 1)
        cv2.putText(frame, str('accuracy=')+str(np.round(match,2)), (left + 6, bottom - 60), font, 1.0, (255, 255, 255), 1)

    # Покажем это все на экране
    cv2.imshow('Video', frame)

    # Обозначим "q" как кнопку выхода из программы 
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Откроем окнов вебкмеры
video_capture.release()
cv2.destroyAllWindows()